# 10. 단어 분리하기(Byte Pair Encoding, BPE)

- 머신 러닝을 이용한 자연어 처리의 최종 목표는 기계가 사람 이상의 성능을 내는 것을 기대하는 것입니다. 그런데 기계에게 아무리 많은 단어를 학습시켜도, 세상의 모든 단어를 알려줄 수는 없는 노릇입니다. 그리고 더 많은 단어를 알려주려고 하면 그만큼 계산 비용도 늘어난다는 부담이 있습니다.


- 기계가 훈련 단계에서 학습한 단어들의 집합을 단어 집합(vocabulary)이라고 합니다. 그리고 테스트 단계에서 기계가 미처 배우지 못한 모르는 단어가 등장하면 그 단어를 단어 집합에 없는 단어란 의미에서 OOV(Out-Of-Vocabulary) 또는 UNK(Unknown Token)라고 표현합니다. 기계가 문제를 풀 때, 모르는 단어가 등장하면 (사람도 마찬가지지만) 주어진 문제를 푸는 것이 훨씬 어려워집니다. 이와 같이 모르는 단어로 인해 문제를 제대로 풀지 못하는 상황을 OOV 문제라고 합니다.


- **단어 분리(Subword segmenation) 작업**은 하나의 단어는 (단어보다 작은 단위의) 의미있는 여러 내부 단어들(subwords)의 조합으로 구성된 경우가 많기 때문에, <ins>하나의 단어를 여러 내부 단어로 분리해서 단어를 이해해보겠다는 의도를 가진 전처리 작업</ins>입니다. 실제로, 언어의 특성에 따라 영어권 언어나 한국어는 단어 분리를 시도했을 때 어느정도 의미있는 단위로 나누는 것이 가능합니다. 이 책에서는 이런 작업을 하는 토크나이저를 단어 **분리 토크나이저**라고 명명하겠습니다.


- 단어 분리는 기계가 아직 배운 적이 없는 단어에 대해 어느 정도 대처할 수 있도록 하며, 기계 번역 등에서 주요 전처리로 사용되고 있습니다. 지금부터 OOV 문제를 완화하는 대표적인 단어 분리 토크나이저인 BPE(Byte Pair Encoding) 알고리즘과 실무에서 사용할 수 있는 단어 분리 토크나이저 구현체인 센텐스피스(Sentencepiece)를 소개합니다.

## 1) BPE(Byte Pair Encoding)
- 1994년에 제안된 데이터 압축 알고리즘 $\Longrightarrow$ 자연어 처리의 단어 분리 알고리즘으로 응용
- BPE은 기본적으로 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행

aaabdaaabac  
$\Longrightarrow$ ZabdZabac (Z=aa)  
$\Longrightarrow$ ZYdZYac (Y=ab, Z=aa)  
$\Longrightarrow$ XdXac (X=ZY, Y=ab, Z=aa) [최종결과]

## 2) 자연어 처리에서의 BPE(Byte Pair Encoding)

논문 : https://arxiv.org/pdf/1508.07909.pdf

자연어 처리에서의 BPE는 단어 분리(word segmentation) 알고리즘입니다. 기존에 있던 단어를 분리한다는 의미입니다. BPE을 요약하면, 글자(charcter) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용합니다. 우선 훈련 데이터에 있는 단어들을 모든 글자(characters) 또는 유니코드(unicode) 단위로 단어 집합(vocabulary)를 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합합니다.

BPE을 자연어 처리에 사용한다고 제안한 논문(Sennrich et al. (2016))에서 이미 BPE의 코드를 공개하였기 때문에, 바로 파이썬 실습이 가능합니다. 코드 실습을 진행하기 전에 육안으로 확인할 수 있는 간단한 예를 들어보겠습니다.

### 기존의 접근
어떤 훈련 데이터로부터 각 단어들의 빈도수를 카운트했다고 해보겠습니다. 그리고 각 단어와 각 단어의 빈도수가 기록되어져 있는 해당 결과는 임의로 딕셔너리(dictionary)란 이름을 붙였습니다.

- 훈련 데이터에 있는 단어와 등장 빈도수
    - low : 5, lower : 2, newest : 6, widest : 3

- vocabulary(단어집합)
    - low, lower, newest, widest
    
이 경우 테스트 과정에서 'lowest'란 단어가 등장한다면 기계는 이 단어를 학습한 적이 없으므로 해당 단어에 대해서 제대로 대응하지 못하는 OOV 문제가 발생합니다. 그렇다면 BPE를 적용한다면 어떨까요?

### BPE 알고리즘을 사용한 경우
- 우선 딕셔너리의 모든 단어들을 글자(character) 단위로 분리
- 이제부터 딕셔너리는 자신 또한 업데이트되며, 앞으로 단어 집합을 업데이트하기 위해 지속적으로 참고되는 참고 자료의 역할을 함

- dictionary
    - l o w : 5,  l o w e r : 2,  n e w e s t : 6,  w i d e s t : 3

- vocabulary
    - l, o, w, e, r, n, w, s, t, i, d

- **BPE의 특징** : 알고리즘의 동작을 <ins>몇 회 반복(iteration)할 것인지</ins>를 사용자가 정함
- 여기서는 총 10회를 수행한다고 가정합니다. 다시 말해 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 총 10회 반복합니다. 위의 딕셔너리에 따르면 빈도수가 현재 가장 높은 유니그램의 쌍은 (e, s)입니다.

#### 1회 - 딕셔너리를 참고로 하였을 때 빈도수가 9로 가장 높은 (e, s)의 쌍을 es로 통합합니다.
- dictionary update!
    - l o w : 5,
    - l o w e r : 2,
    - n e w es t : 6,
    - w i d es t : 3

- vocabulary update!
    - l, o, w, e, r, n, w, s, t, i, d, es

#### 2회 - 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합합니다.
- dictionary update!
    - l o w : 5,
    - l o w e r : 2,
    - n e w est : 6,
    - w i d est : 3
    
- vocabulary update!
    - l, o, w, e, r, n, w, s, t, i, d, es, est

#### 3회 - 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합합니다.

- dictionary update!
    - lo w : 5,
    - lo w e r : 2,
    - n e w est : 6,
    - w i d est : 3
    
- vocabulary update!
    - l, o, w, e, r, n, w, s, t, i, d, es, est, lo

이와 같은 방식으로 총 10회 반복하였을 때 얻은 딕셔너리와 단어 집합은 아래와 같습니다.

- dictionary update!
    - low : 5,
    - low e r : 2,
    - newest : 6,
    - widest : 3
    
- vocabulary update!
    - l, o, w, e, r, n, w, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest

테스트 과정에서 'lowest'란 단어가 등장한다면, 기존에는 OOV에 해당되는 단어가 되었겠지만 BPE 알고리즘을 사용한 위의 단어 집합에서는 더 이상 'lowest'는 OOV가 아닙니다. 기계는 우선 'lowest'를 전부 글자 단위로 분할합니다. 즉, 'l, o, w, e, s, t'가 됩니다. 그리고 기계는 위의 단어 집합을 참고로 하여 'low'와 'est'를 찾아냅니다. 즉, 'lowest'를 기계는 'low'와 'est' 두 단어로 인코딩합니다. 그리고 이 두 단어는 둘 다 단어 집합에 있는 단어이므로 OOV가 아닙니다.

### 코드 실습하기

In [1]:
import re, collections

num_merges = 10 # BPE을 몇 회 수행할 것인지

vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

In [2]:
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best) # 글자들의 통합 과정을 보여줌

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


- e와 s의 쌍은 초기 단어 집합에서 총 9회 등장했습니다. 그렇기 때문에 es로 통합됩니다.
- 그 다음으로는 es와 t의 쌍을, 그 다음으로는 est와 </w>의 쌍을 통합시킵니다.
- 빈도수가 가장 높은 순서대로 통합하는 이 과정을 총 num_merges회 반복한 것입니다.

## 2) WPM(Wordpiece Model)
- 구글이 WPM 오픈소스 공개X

## 3) 센텐스피스(Sentencepiece)
- 향후 wikidocs로 추가 예정이라고 함

논문 : https://arxiv.org/pdf/1808.06226.pdf
센텐스피스 깃허브 : https://github.com/google/sentencepiece

결론적으로 실무에서 단어 분리를 위해서 어떤 구현체를 사용해야 하냐고 묻는다면, 구글의 센텐스피스(Sentencepiece)를 사용합니다. 구글은 BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 센텐스피스를 깃허브에 공개하였습니다. 기존의 BPE 알고리즘 논문 저자 또한 BPE 코드를 깃허브에 공개하기는 했지만, 이를 실무에 사용하기에는 속도가 매우 느리므로 센텐스피스를 사용하는 것을 권장합니다.

센텐스피스의 이점은 또 있습니다. 단어 분리 알고리즘을 사용하기 위해서, 데이터에 단어 토큰화를 먼저 진행한 상태여야 한다면 이 단어 분리 알고리즘을 모든 언어에 사용하는 것은 쉽지 않습니다. 영어와 달리 한국어와 같은 언어는 단어 토큰화부터가 쉽지 않기 때문입니다. 그런데, 이런 사전 토큰화 작업(pretokenization)없이 전처리를 하지 않은 데이터(raw data)에 바로 단어 분리 토크나이저를 사용할 수 있다면, 이 토크나이저는 그 어떤 언어에도 적용할 수 있는 토크나이저가 될 것입니다. 센텐스피스는 이 이점을 살려서 구현되었습니다. 센텐스피스는 사전 토큰화 작업없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않습니다.